# Extra dependencies

## Find Files of interest

In [2]:
import fsspec

data_dir = "s3://nasa-waterinsight/NLDAS3/forcing/daily/"

# Use fsspec to list files in the S3 bucket
fs = fsspec.filesystem("s3", anon=True)
files = fs.glob(data_dir + "**/*.nc")

## Produce a virtual dataset from the list of files

In [4]:
from virtualizarr import open_virtual_dataset, open_virtual_mfdataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry

import obstore

bucket = "s3://nasa-waterinsight"
store = obstore.store.from_url(bucket, region="us-west-2", skip_signature=True)

registry = ObjectStoreRegistry({bucket:store})

parser = HDFParser()

urls = ['s3://'+file for file in files]
vds = open_virtual_mfdataset(urls, parser=parser, registry=registry, parallel='lithops')

2025-08-01 20:32:26,002 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.11
2025-08-01 20:32:26,004 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-01 20:32:26,004 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-01 20:32:26,051 [INFO] config.py:139 -- Lithops v3.6.1 - Python3.11
2025-08-01 20:32:26,052 [INFO] localhost.py:39 -- Localhost storage client created
2025-08-01 20:32:26,052 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-08-01 20:32:26,105 [INFO] invokers.py:119 -- ExecutorID d62bfe-1 | JobID M000 - Selected Runtime: python 
2025-08-01 20:32:27,621 [INFO] invokers.py:186 -- ExecutorID d62bfe-1 | JobID M000 - Starting function invocation: _open() - Total: 8399 activations
2025-08-01 20:34:45,252 [INFO] invokers.py:225 -- ExecutorID d62bfe-1 | JobID M000 - View execution logs at /tmp/lithops-root/logs/d62bfe-1-M000.log
2025-08-01 20:34:45,500 [INFO] executors.py:494 -- ExecutorID d62bfe-1 - Getting results from 8399 fu

    0%|          | 0/8399  

2025-08-01 22:07:17,678 [INFO] executors.py:618 -- ExecutorID d62bfe-1 - Cleaning temporary data
/opt/conda/lib/python3.11/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [5]:
vds

<xarray.Dataset> Size: 26TB
Dimensions:   (time: 8399, lat: 6500, lon: 11700)
Coordinates:
  * lon       (lon) float64 94kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
  * lat       (lat) float64 52kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
  * time      (time) datetime64[ns] 67kB 2001-01-02 2001-01-03 ... 2024-01-01
Data variables:
    Tair      (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    Tair_min  (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    Tair_max  (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    Qair      (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    PSurf     (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    LWdown    (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    SWdown    (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    Wind_N    (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    Wind_E    (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
    Rainf     (time, lat, lon) float32 3TB ManifestArray<shape=(8399, 6500, 1...
Attributes: (12/17)
    missing_value:          -9999.0
    time_definition:        daily
    shortname:              NLDAS_FOR0010_D_3.0
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    version:                3.0 beta
    institution:            NASA GSFC
    ...                     ...
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  7.005000114440918
    SOUTH_WEST_CORNER_LON:  -168.9949951171875
    DX:                     0.009999999776482582
    DY:                     0.009999999776482582

## Write (commit) the virtual dataset into icechunk

In [9]:
import icechunk

In [10]:
storage = icechunk.s3_storage(
    bucket='nasa-veda-scratch',
    prefix=f"jbusecke/NLDAS3/",
    anonymous=False,
    from_env=True,
)

config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        "s3://nasa-waterinsight/NLDAS3/forcing/daily/202201/",
        icechunk.s3_store(region='us-west-2')
    )
)

virtual_credentials = icechunk.containers_credentials(
    {
        "s3://nasa-waterinsight/NLDAS3/forcing/daily/202201/": icechunk.s3_anonymous_credentials()
    }
)
    
repo = icechunk.Repository.open_or_create(
    storage=storage,
    config=config,
    authorize_virtual_chunk_access=virtual_credentials,
)

session = repo.writable_session('main')
vds.vz.to_icechunk(session.store)
session.commit('First Try full dataset')

'GXR61CSGXB589T1JPGN0'

## Read the data back into xarray

In [1]:
%time

# self contained read that works on the hub

import icechunk
import xarray as xr
import zarr
# zarr.config.set({'async.concurrency':128}) # does this improve compute speed for the timeseries


storage = icechunk.s3_storage(
    bucket='nasa-veda-scratch',
    prefix=f"jbusecke/nasa-waterinsight-test-full/",
    anonymous=False,
    from_env=True,
)

config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        "s3://nasa-waterinsight/NLDAS3/forcing/daily/",
        icechunk.s3_store(region='us-west-2')
    )
)

virtual_credentials = icechunk.containers_credentials(
    {
        "s3://nasa-waterinsight/NLDAS3/forcing/daily/202201/": icechunk.s3_anonymous_credentials()
    }
)
    
repo = icechunk.Repository.open(
    storage=storage,
    config=config,
    authorize_virtual_chunk_access=virtual_credentials,
)

session = repo.readonly_session('main')
ds = xr.open_zarr(session.store, consolidated=False, zarr_version=3)
ds

CPU times: user 0 ns, sys: 3 μs, total: 3 μs
Wall time: 5.01 μs


/tmp/ipykernel_804816/3665244252.py:39: FutureWarning: zarr_version is deprecated, use zarr_format
  ds = xr.open_zarr(session.store, consolidated=False, zarr_version=3)
/opt/conda/lib/python3.11/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


<xarray.Dataset> Size: 51TB
Dimensions:   (time: 8399, lat: 6500, lon: 11700)
Coordinates:
  * lon       (lon) float64 94kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
  * time      (time) datetime64[ns] 67kB 2001-01-02 2001-01-03 ... 2024-01-01
  * lat       (lat) float64 52kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
Data variables:
    LWdown    (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Qair      (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    PSurf     (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Tair_max  (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Tair      (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Wind_N    (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Rainf     (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    SWdown    (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Tair_min  (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
    Wind_E    (time, lat, lon) float64 5TB dask.array<chunksize=(1, 500, 900), meta=np.ndarray>
Attributes: (12/17)
    missing_value:          -9999.0
    time_definition:        daily
    shortname:              NLDAS_FOR0010_D_3.0
    title:                  NLDAS Forcing Data L4 Daily 0.01 x 0.01 degree V3...
    version:                3.0 beta
    institution:            NASA GSFC
    ...                     ...
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  7.005000114440918
    SOUTH_WEST_CORNER_LON:  -168.9949951171875
    DX:                     0.009999999776482582
    DY:                     0.009999999776482582

## Testing a timeseries at a single location

In [22]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    timeseries = ds.sel(lon=-74.00, lat=40.71, method='nearest').load() # load timeseries close to NYC into memory

[########################################] | 100% Completed | 145.31 s


## Failed tests
### Dask distributed

Fails with a deserialization error. 

In [2]:
from distributed import Client
client = Client()
client

timeseries = ds.sel(lon=-74.00, lat=40.71, method='nearest').load() # load timeseries close to NYC into memory

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jbusecke/proxy/8787/status,
Dashboard: /user/jbusecke/proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 60.62 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42319,Workers: 0
Dashboard: /user/jbusecke/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43203,Total threads: 4
Dashboard: /user/jbusecke/proxy/35139/status,Memory: 15.16 GiB
Nanny: tcp://127.0.0.1:33509,


2025-08-01 22:33:15,140 - distributed.scheduler - ERROR - Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments



In [ ]:
timeseries.plot()

### Testing performance impact of concurrency

I changed the `async.concurrency` from 10 to 128 with no noticeable difference
- 142.56 s before changing zarr config
- 145.31 s with concurrency 128